In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import BertTokenizer
import torch
from torch import nn
from transformers import BertModel
import numpy as np
from torch.optim import Adam
from tqdm import tqdm
from gensim.models import Word2Vec
from sklearn.preprocessing import scale
from sklearn import model_selection, svm
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os

df = pd.read_csv("/kaggle/input/feedback-prize-2021/train.csv")
df

In [ ]:
labels = {'Lead':0,
          'Position':1,
          'Claim':2,
          'Concluding Statement':3,
          'Evidence':4,
          'Counterclaim':5,
          'Rebuttal':6
          }
prev_label = {'Lead':0,
          'Position':1,
          'Claim':2,
          'Concluding Statement':3,
          'Evidence':4,
          'Counterclaim':5,
          'Rebuttal':6,
          'Start':7
          }

In [ ]:
sequence = df['predictionstring']
sequence = sequence.apply(lambda x: x.split())
sequence = sequence.apply(lambda x: list(map(int, x)))
sequence
df_new = df
start, end = [],[]
for i in range(len(sequence)):
    start.append(sequence[i][0])
    end.append(sequence[i][-1])
df_new['Start'] = start
df_new['End'] = end

prev = [7]

for i in range(1, len(sequence)):
    if df_new['End'][i-1] < df_new['Start'][i]:
        prev.append(prev_label[df['discourse_type'][i-1]])
    else:
        prev.append(7)

In [ ]:
df_new['one_hot'] = prev
df_new

In [ ]:
from gensim.utils import simple_preprocess
# Tokenize the text column to get the new column 'tokenized_text'
df_new['tokenized_text'] = [simple_preprocess(line, deacc=True) for line in df_new['discourse_text']] 
print(df_new['tokenized_text'].head(10))

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_new, test_size = 0.2, stratify = df_new['discourse_type'])

In [ ]:
x_train = train['tokenized_text']
y_train = train['discourse_type']
x_test = test['tokenized_text']
y_test = test['discourse_type']

In [ ]:
dim = 200
w2v_model = Word2Vec(vector_size = dim)
w2v_model.build_vocab(x_train)
w2v_model.train(x_train, total_examples = w2v_model.corpus_count, epochs = 20)

In [ ]:
#Build word vector for training set by using the average value of all word vectors in the tweet, then scale
def wordvector(text, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in text:
        try:
            vec += w2v_model.wv[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [ ]:
train_vec = np.concatenate([wordvector(text, dim) for text in x_train])
train_vec = scale(train_vec)

In [ ]:
w2v_model.train(x_test, total_examples =  len(x_test), epochs = w2v_model.epochs)

In [ ]:
test_vec = np.concatenate([wordvector(text, dim) for text in x_test])
test_vec = scale(test_vec)

In [ ]:
# word2vec svm
svm_clf = svm.SVC(kernel='rbf',verbose=True)
svm_clf.fit(train_vec, y_train)

In [ ]:
svm_pred = svm_clf.predict(test_vec)
svm_accu = metrics.accuracy_score(svm_pred, y_test)
svm_f1 = metrics.f1_score(svm_pred, y_test)

In [ ]:
# word2vec random forest
from sklearn.ensemble import RandomForestClassifier as RFC
rfc_clf = RFC(n_estimators = 100, oob_score = True)
rfc_clf.fit(train_vec, y_train)

In [ ]:
rfc_pred = rfc_clf.predict(test_vec)
rfc_accu = metrics.accuracy_score(rfc_pred, y_test)
rfc_f1 = metrics.f1_score(rfc_pred, y_test)

In [ ]:
scaler1 = MinMaxScaler(copy = False)
scaler1.fit(train_vec)
scaler2 = MinMaxScaler(copy = False)
scaler2.fit(test_vec)
train_01 = scaler1.transform(train_vec)
test_01 = scaler2.transform(test_vec)

In [ ]:
# word2vec naivebayes
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(train_01,y_train)

In [ ]:
mnb_pred = mnb.predict(test_01)
mnb_accu = metrics.accuracy_score(mnb_pred, y_test)
mnb_f1 = metrics.f1_score(mnb_pred, y_test)

In [ ]:
metrics.ConfusionMatrixDisplay.from_predictions(rfc_pred, y_test)
plt.show()

In [ ]:
metrics.ConfusionMatrixDisplay.from_predictions(svm_pred, y_test)
plt.show()

In [ ]:
metrics.ConfusionMatrixDisplay.from_predictions(mnb_pred, y_test)
plt.show()